# get the data

In [48]:
%load_ext autoreload
%autoreload 2

In [49]:
import os
import requests 
from pathlib import Path
import zipfile
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from typing import List,Dict,Tuple
from tqdm import tqdm

In [50]:
data=Path("data/")
image_path=data/"pizza_steak_sushi"

#lets make the directory......
if image_path.is_dir():
    print("the directory already exists")
else:
    image_path.mkdir(parents=True,exist_ok=True)
    print("created the directory...")

#lets load the images into that directory   "
request=requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
#image_file=data/"pizza-steak-sushi"
with open(data/"pizza_steak_sushi.zip","wb") as f:
    f.write(request.content)

the directory already exists


In [51]:
#lets extract the ziped file
with zipfile.ZipFile(data/"pizza_steak_sushi.zip","r") as zip_file:
    zip_file.extractall(image_path)

In [52]:
train_dir=image_path/"train"
test_dir=image_path/"test"

In [53]:
#remove the zipped file
os.remove(data/"pizza_steak_sushi.zip")

In [54]:
train_dir,test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

# 1 create datasets,data loader and transforms

In [55]:
#transforms
train_transform=transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

test_transform=transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [56]:
train_dataset=ImageFolder(
    root=train_dir,
    transform=train_transform,
)
test_dataset=ImageFolder(
    root=test_dir,
    transform=test_transform
)

In [57]:
class_names=train_dataset.classes
class_names

['pizza', 'steak', 'sushi']

In [58]:
class_idx=train_dataset.class_to_idx
class_idx

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [59]:
print(train_dataset)
print(test_dataset)

Dataset ImageFolder
    Number of datapoints: 225
    Root location: data\pizza_steak_sushi\train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data\pizza_steak_sushi\test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


In [60]:
batch_size=32

train_dataloader=DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
test_dataloader=DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
)

In [61]:
image,label=next(iter(train_dataloader))
image.shape,label.shape

(torch.Size([32, 3, 64, 64]), torch.Size([32]))

In [62]:
data_transform=transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [63]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [64]:
%%writefile going_modular/data_setup.py
"""makes training and testing dataloader"""
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

def create_dataloader(train_dir:str,
                     test_dir:str,
                    transform:transforms.Compose,
                     batch_size:int
                     ):
    
    """takes in filepath and returns train data loader,test data loader and numbers of classes"""
 
    train_dataset=ImageFolder(
        root=train_dir,
        transform=transform,
        )
    
    test_dataset=ImageFolder(
        root=test_dir,
        transform=transform
    )

    class_names=train_dataset.classes
    
    train_dataloader=DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=True,
    )
    
    test_dataloader=DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        shuffle=False,
        pin_memory=True
    )

    return train_dataloader,test_dataloader,class_names

Overwriting going_modular/data_setup.py


## **lets test the module we just created**

In [65]:
train_dir,test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [66]:
data_transform

Compose(
    Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
)

In [67]:
from going_modular import data_setup

batch_size=32

local_trainloader,local_testloader,class_names=data_setup.create_dataloader(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=batch_size
)


In [68]:
local_trainloader,local_testloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x144810f6190>,
 ['pizza', 'steak', 'sushi'])

# lets the save the datasets,data loader

# 2 make the model

In [69]:
class TinnyVGG(nn.Module):
    """ makes tunny vgg model """
    def __init__(self,input_shape:int,
                 n_hidden:int,
                output_shape:int):
        super().__init__()
        
        self.block1=nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                             out_channels=n_hidden,
                             kernel_size=3,
                             stride=1,
                             padding=0),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=n_hidden,
                     out_channels=n_hidden,
                     kernel_size=3,
                     stride=1,
                     padding=0),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2,
                        stride=2,
                        padding=0)
        )
        self.block2=nn.Sequential(
            nn.Conv2d(in_channels=n_hidden,
                      out_channels=n_hidden,
                     kernel_size=3,
                     stride=1,
                     padding=0),
            
             nn.ReLU(),
            
             nn.Conv2d(in_channels=n_hidden,
                     out_channels=n_hidden,
                     kernel_size=3,
                     stride=1,
                     padding=0),
            nn.ReLU(),
           
            nn.MaxPool2d(kernel_size=2,
                      stride=2,
                      padding=0)
        )
        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=n_hidden*13*13,
                      out_features=output_shape)
        )
    def forward(self,x:torch.Tensor):
        #print(f"initial shape: {x.shape}")
        x=self.block1(x)
        #print(f"after block_1: {x.shape}")
        x=self.block2(x)
        #print(f"after block_2: {x.shape}")
        x=self.classifier(x)
        #print(f"final shape:{x.shape} ")
        return x

# lets save this class into a scrips

In [70]:
%%writefile going_modular/model_builder.py
import torch
import torch.nn as nn

class TinnyVGG(nn.Module):
    """ makes tunny vgg model """
    def __init__(self,input_shape:int,
                 n_hidden:int,
                output_shape:int):
        super().__init__()
        
        self.block1=nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                             out_channels=n_hidden,
                             kernel_size=3,
                             stride=1,
                             padding=0),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=n_hidden,
                     out_channels=n_hidden,
                     kernel_size=3,
                     stride=1,
                     padding=0),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2,
                        stride=2,
                        padding=0)
        )
        self.block2=nn.Sequential(
            nn.Conv2d(in_channels=n_hidden,
                      out_channels=n_hidden,
                     kernel_size=3,
                     stride=1,
                     padding=0),
            
             nn.ReLU(),
            
             nn.Conv2d(in_channels=n_hidden,
                     out_channels=n_hidden,
                     kernel_size=3,
                     stride=1,
                     padding=0),
            nn.ReLU(),
           
            nn.MaxPool2d(kernel_size=2,
                      stride=2,
                      padding=0)
        )
        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=n_hidden*13*13,
                      out_features=output_shape)
        )
    def forward(self,x:torch.Tensor):
        #print(f"initial shape: {x.shape}")
        x=self.block1(x)
        #print(f"after block_1: {x.shape}")
        x=self.block2(x)
        #print(f"after block_2: {x.shape}")
        x=self.classifier(x)
        #print(f"final shape:{x.shape} ")
        return x

Overwriting going_modular/model_builder.py


# l**ets test our model builder**

In [71]:
from going_modular.model_builder import TinnyVGG

dummy_model=TinnyVGG(
    input_shape=1,
    output_shape=10,
    n_hidden=10
)
dummy_model.state_dict

<bound method Module.state_dict of TinnyVGG(
  (block1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=10, bias=True)
  )
)>

In [72]:
# lets check for some shape
image,label=next(iter(train_dataloader))
image.shape,label.shape

(torch.Size([32, 3, 64, 64]), torch.Size([32]))

In [73]:
class_names

['pizza', 'steak', 'sushi']

In [74]:
model_v9=TinnyVGG(input_shape=3,
                   n_hidden=10,
                   output_shape=len(class_names)).to(device)
model_v9.state_dict

<bound method Module.state_dict of TinnyVGG(
  (block1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)>

# 3 train step,test step and train all 

In [75]:
def train_step(
              model:nn.Module,
              train_data:torch.utils.data.DataLoader,
              loss_fn:torch.nn,
              optimizer:torch.optim.Optimizer,
              device:torch.device,
              )->Tuple[float,float]:
    
    train_loss,train_acc=0,0
    model.train()
    
    for batch,(x,y) in enumerate(train_data):
        x,y=x.to(device),y.to(device)
        pred_logits=model(x)
        
        preds=torch.argmax(torch.softmax(pred_logits,dim=1),dim=1)
        loss=loss_fn(pred_logits,y).item()
        
        train_loss+=loss
        train_acc+=(preds==y).sum().item()/len(train_data)
    
        #lets zero grad the gradient
        optimizer.zero_grad()
        #lets backpropagate
        loss.backward()
        #lets update the parameter
        optimizer.step()
        
    train_loss/=len(train_data)
    train_acc/=len(train_data)
    
    return train_loss,train_acc

In [76]:
def test_step(model:nn.Module,
             test_data:torch.utils.data.DataLoader,
             loss_fn:torch.nn,
             device:torch.device="cpu")->Tuple[float,float]:
    
    test_loss,test_acc=0,0
    model.eval()
    with torch.inference_mode():
        for batch,(x,y) in enumerate(test_data):
        
            x,y=x.to(device),y.to(device)
            y_pred_logits=model(x)
            
            y_pred=torch.argmax(torch.softmax(y_pred_logits,dim=1),dim=1)
            loss=loss_fn(y_pred_logits,y).item()
            
            test_loss+=loss
            test_acc=(y==y_pred).sum().item()/len(test_data)
    test_loss/=len(test_data)
    test_acc/=len(test_data)
    return test_loss,test_acc

In [77]:
def train_model(model:nn.Module,
               train_data:torch.utils.data.DataLoader,
               test_data:torch.utils.data.DataLoader,
               loss_fn:torch.nn,
                optimizer:torch.optim.Optimizer,
                epochs:int,
               device:torch.device):
    #TRAIN MDOEL
    results={"train_loss":[],
            "train_acc":[],
            "test_loss":[],
            "test_acc":[]}
    
    train_loss,train_acc=0,0
    for epoch in tqdm(range(epochs)):
        train_loss,train_acc=train_step(model=model,
                                       train_data=train_data,
                                       loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        device=device
                                       )
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        
    #TEST THE MDODEL
    test_loss,test_acc=0,0
    test_loss,test_acc=model(model=test_data,
                             test_data=test_data,
                            loss_fn=loss_fn,
                            device=device
                            )
    return results

# lets save this functions as module

In [96]:
%%writefile going_modular/engine.py 
import torch
import torch.nn as nn
from typing import List,Tuple
from tqdm import tqdm

def train_step(
              model:nn.Module,
              train_data:torch.utils.data.DataLoader,
              loss_fn:torch.nn,
              optimizer:torch.optim.Optimizer,
              device:torch.device,
              )->Tuple[float,float]:
    
    train_loss,train_acc=0,0
    model.train()
    
    for batch,(x,y) in enumerate(train_data):
        x,y=x.to(device),y.to(device)
        pred_logits=model(x)
        
        preds=torch.argmax(torch.softmax(pred_logits,dim=1),dim=1)
        loss=loss_fn(pred_logits,y)
        
        train_loss+=loss.item()
        train_acc+=(preds==y).sum().item()/len(train_data)
    
        #lets zero grad the gradient
        optimizer.zero_grad()
        #lets backpropagate
        loss.backward()
        #lets update the parameter
        optimizer.step()
        
    train_loss/=len(train_data)
    train_acc/=len(train_data)
    
    return train_loss,train_acc
    
def test_step(model:nn.Module,
             test_data:torch.utils.data.DataLoader,
             loss_fn:torch.nn,
             device:torch.device="cpu")->Tuple[float,float]:
    
    test_loss,test_acc=0,0
    model.eval()
    with torch.inference_mode():
        for batch,(x,y) in enumerate(test_data):
        
            x,y=x.to(device),y.to(device)
            y_pred_logits=model(x)
            
            y_pred=torch.argmax(torch.softmax(y_pred_logits,dim=1),dim=1)
            loss=loss_fn(y_pred_logits,y).item()
            
            test_loss+=loss
            test_acc=(y==y_pred).sum().item()/len(test_data)
    test_loss/=len(test_data)
    test_acc/=len(test_data)
    return test_loss,test_acc

def train_model(model:nn.Module,
               train_data:torch.utils.data.DataLoader,
               test_data:torch.utils.data.DataLoader,
               loss_fn:torch.nn,
                optimizer:torch.optim.Optimizer,
                epochs:int,
               device:torch.device):
    #TRAIN MDOEL
    results={"train_loss":[],
            "train_acc":[],
            "test_loss":[],
            "test_acc":[]}
    
    train_loss,train_acc=0,0
    for epoch in tqdm(range(epochs)):
        train_loss,train_acc=train_step(model=model,
                                       train_data=train_data,
                                       loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        device=device
                                       )
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        
        #TEST THE MDODEL
        test_loss,test_acc=0,0
        test_loss,test_acc=test_step(model=model,
                             test_data=test_data,
                            loss_fn=loss_fn,
                            device=device
                            )
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    return results

Overwriting going_modular/engine.py


In [79]:
from going_modular.engine import train_model

In [80]:
dummy_results=train_model(
    model=dummy_model,
    train_data=,
    test_data=,
    l
)

SyntaxError: invalid syntax (3721115479.py, line 3)

# 3 save the model

In [111]:
def save_model(model:nn.Module,
              target:str,
              model_name:str):
    
    target_path=Path(target)
    target_path.mkdir(parents_ok=True,exists=True)
    
    torch.save(model,target_path)
    #assert the externsion of the model
    
    assert model_name.endswith(".pt") or model_name.endswith(".pth"),"model extension should be .pt or .pth"
    model_path=target_path/model_name  #created the directory

    print(f"saving the model into {model_path}")
    torch.save(model.state_dict(),model_path)

# lets modularize the saving function

In [113]:
%%writefile going_modular/utils.py
import torch
import torch.nn as nn
from pathlib import Path

def save_model(model:nn.Module,
              target:str,
              model_name:str):
    
    target_path=Path(target)
    target_path.mkdir(parents=True,exist_ok=True)
    
    #torch.save(model,target_path)
    #assert the externsion of the model
    
    assert model_name.endswith(".pt") or model_name.endswith(".pth"),"model extension should be .pt or .pth"
    model_path=target_path/model_name  #created the directory

    print(f"saving the model into {model_path}")
    torch.save(model.state_dict(),f=str(model_path))

Overwriting going_modular/utils.py


# train and evaluate model

In [102]:
train_dir,test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [103]:
data_transform

Compose(
    Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
)

In [122]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from going_modular.data_setup import create_dataloader
from going_modular.engine import train_model
from going_modular.model_builder import TinnyVGG
from going_modular.utils import save_model
from timeit import default_timer as timer
import torch.nn as nn

batch_size=32
learning_rate=0.01
n_neurons=10

#lets define the data loader-->data sets
train_dataloader,test_dataloader,class_names=create_dataloader(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=batch_size
)

#make the model
model_v1=TinnyVGG(
    input_shape=3,
    output_shape=len(class_names),
    n_hidden=n_neurons
)

#lets set up the parameters
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model_v1.parameters(),lr=learning_rate)

start_time=timer()
#lets train our model
epochs=5
results=train_model(
    model=model_v1,
    train_data=train_dataloader,
    test_data=test_dataloader,
    loss_fn=loss,
    optimizer=optimizer,
    epochs=epochs,
    device=device
)
end_time=timer()
print(f"total amount of time taken to train the model is: {end_time-start_time}")
print(results)

#lets save the model
save_model(model_v1,target="models",model_name="tinnyVGG_model.pt")

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:15<00:00,  3.11s/it]

total amount of time taken to train the model is: 15.532730800000081
{'train_loss': [1.099340483546257, 1.1030633747577667, 1.0991303473711014, 1.101421520113945, 1.1000436544418335], 'train_acc': [1.21875, 1.125, 1.265625, 1.125, 1.171875], 'test_loss': [1.0827259222666423, 1.0977166891098022, 1.0840776364008586, 1.099848707516988, 1.0856244564056396], 'test_acc': [1.222222222222222, 0.0, 1.222222222222222, 0.0, 1.222222222222222]}
saving the model into models\tinnyVGG_model.pt


# lets save the training and saving of model as script

In [131]:
train_dir,test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [140]:
%%writefile going_modular/train.py


from data_setup import create_dataloader
from engine import train_model
from model_builder import TinnyVGG
from utils import save_model
from timeit import default_timer as timer
import torch.nn as nn
from torchvision import transforms
import torch

torch.manual_seed(42)
torch.cuda.manual_seed(42)


batch_size=32
learning_rate=0.01
n_neurons=10

train_dir="data/pizza_steak_sushi/train"
test_dir="data/pizza_steak_sushi/test"

data_transform=transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

device="cuda" if torch.cuda.is_available() else "cpu"


#lets define the data loader-->data sets
train_dataloader,test_dataloader,class_names=create_dataloader(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=batch_size
)

#make the model
model_v1=TinnyVGG(
    input_shape=3,
    output_shape=len(class_names),
    n_hidden=n_neurons
)

#lets set up the parameters
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model_v1.parameters(),lr=learning_rate)

start_time=timer()
#lets train our model
epochs=5
results=train_model(
    model=model_v1,
    train_data=train_dataloader,
    test_data=test_dataloader,
    loss_fn=loss,
    optimizer=optimizer,
    epochs=epochs,
    device=device
)
end_time=timer()
print(f"total amount of time taken to train the model is: {end_time-start_time}")
print(results)

#lets save the model
save_model(model_v1,target="models",model_name="tinnyVGG_model.pt")

Overwriting going_modular/train.py


# train model with one line of code 

In [141]:
!python going_modular/train.py

total amount of time taken to train the model is: 7.3340724999999996
{'train_loss': [1.099340483546257, 1.1030633747577667, 1.0991303473711014, 1.101421520113945, 1.1000436544418335], 'train_acc': [1.21875, 1.125, 1.265625, 1.125, 1.171875], 'test_loss': [1.0827259222666423, 1.0977166891098022, 1.0840776364008586, 1.099848707516988, 1.0856244564056396], 'test_acc': [1.222222222222222, 0.0, 1.222222222222222, 0.0, 1.222222222222222]}
saving the model into models\tinnyVGG_model.pt



100%|##########| 5/5 [00:07<00:00,  1.47s/it]
